# Instrument object
This section shows the majority of the features implemented for the instrument object in McStasScript.

## Initialization
An instrument object is created with the [McStas_instr](../_autosummary/mcstasscript.interface.instr.McStas_instr.rst) or [McXtrace_instr](../_autosummary/mcstasscript.interface.instr.McXtrace_instr.rst) class. When an instrument object is created the only required argument is the name of the instrument which will be used for the instrument filename. There are however a number of keyword arguments that can be used to provide more information and alter the behavior.

| Keyword argument | Type | Default | Description |
| --- | --- | --- | --- |
| author | str |"Python Instrument Generator" | Name that will appear as author in instrument files |
| origin | str |"ESS DMSC" | String that will appear as origin in instrument files |
| input_path | str | "." | Folder which is considered workspace for McStas / McXtrace |
| output_path | str | instrument_name | Name of data folder written by simulation |
| package_path | str | | Can be set to manually specify location of McStas/McXtrace installation |
| executable_path | str |  | Can be set to manually specify location of mcrun/mxrun executable |
| ncount | int, float | 1E6 | Sets the ncount used for simulations |
| mpi | int |  | Sets the number of MPI threads used for simulations |
| force_compile | bool | True | Whether to force compilation before each run or not |
| parameters | ParameterContainer |  | Set of parameters for initialized instrument |

In [ ]:
import mcstasscript as ms

In [ ]:
instrument = ms.McStas_instr("instr_name", author="Mads Bertelsen", origin="DMSC")
instrument_w_settings = ms.McStas_instr("instr_name", ncount=3E6, output_path="new_folder")

### Using settings method
The instrument object has a *setting* method which can update some settings after initialization. The current settings can always be viewed with *show_settings*.

| Keyword argument | Type | Default | Description |
| --- | --- | --- | --- |
| output_path | str | instrument_name | Name of data folder written by simulation |
| package_path | str | | Can be set to manually specify location of McStas/McXtrace installation |
| executable_path | str |  | Can be set to manually specify location of mcrun/mxrun executable |
| ncount | int, float | 1E6 | Sets the ncount used for simulations |
| mpi | int |  | Sets the number of MPI threads used for simulations |
| seed |  |  | Sets the seed of the simulation |
| force_compile | bool | True | Whether to force compilation before each run or not |
| custom_flags | str |  | String with custom flags for mcrun/mxrun command |

In [ ]:
instrument.show_settings()
instrument_w_settings.show_settings()

In [ ]:
instrument.settings(mpi=4, seed=300)
instrument.show_settings()

## Parameters
Instrument parameters can be added with *add_parameters* which returns a parameter object.

In [ ]:
wavelength = instrument.add_parameter("wavelength", comment="Wavelength in AA")
print(wavelength)
wavelength.value = 5
print(wavelength)

### Searching for components and data
McStas have a few keywords for adjusting where to search for data and components. This is typically added right after the parameters, so its natural to include it here in the documentation of the instrument object.

#### Dependency
The DEPENDENCY keyword allows McStas to search for data and software at runtime. There is just one dependency line for an instrument.

In [ ]:
instrument.set_dependency("/dependency/example")

As the instrument won't run unless the dependency is set to a valid path, let's reset it.

In [ ]:
instrument.set_dependency("")

#### Search
The SEARCH keyword allows McStas to search for components before McStas code generation. There can be multiple SEARCH statements in an instrument. It is possible to enable SHELL processing so a system command can be given that should return a path. McStasScript has yet to implement this features fully, so McStasScript won't find components in search paths. For now it can only be used to overwrite existing components with identical input parameters.

In [ ]:
instrument.add_search("/search/example")
instrument.add_search("pwd", SHELL=True)

instrument.show_search()

It is possible to clear all search statements from an instrument with the *clear_search* method.

In [ ]:
instrument.clear_search()
instrument.show_search()

## Initialize section
One of the great advantages for the McStas / McXtrace packages is the initialize section of the instrument where calculations can be performed before the ray-tracing simulation starts. One could for example calculate appropriate angles to reach a certain Bragg peak at a given wavelength. This would involve defining some declare variables, using these in the initialize section and then assigning them as component inputs.

In McStasScript many calculations can be performed directly in Python, and so typically the initialize section is used less, but it is still useful and available through McStasScript.

The instrument object has the method *append_initialize* which adds a line of code to the initialize. This line is copied directly into the instrument file, so it follows C syntax. Remember the semicolon! In addition there is *add_declare_var* to specify the declared variables needed. When declare variables are defined an object is returned which can be used when referring to that variable.

In [ ]:
wavenumber = instrument.add_declare_var("double", "wavenumber")
instrument.append_initialize("wavenumber = 2*PI/wavelength;")

In [ ]:
print(instrument.initialize_section)

## Finally section
The finally section works exactly as the initialize section, but is executed after the ray-tracing simulation. Add a line to it with *append_finally*.

In [ ]:
instrument.append_finally('printf(\"Thanks for using McStasScript!\\n\");')
print(instrument.finally_section)

## Help features
There are a few methods built into the instrument class that helps the user, these are:

- *available_components*
- *component_help*

### available_components
The *available_components* method shows the component categories, and if called with the name of a category, will show all available components in the specified category. The categories can include the work directory if any components are located there.

In [ ]:
instrument.available_components()

In [ ]:
instrument.available_components("optics")

### component_help
The *component_help* method can show the parameters of any component the instrument object knows about, although not necessarily used in the instrument.

In [ ]:
instrument.component_help("Guide")

## Adding components
One adds components to the instrument using *add_component* which takes the name of the component instance for the instrument, followed by the name of the component in the library. When adding a component, a [component](../_autosummary/mcstasscript.helper.mcstas_objects.Component.rst) object is returned, and how these can be manipulated is discussed on the [component object page](component_object.ipynb). Notice that it is not allowed to add two components with the same instance name, meaning rerunning this cell would raise an exception. 

In [ ]:
source = instrument.add_component("source", "Source_div")
source.set_parameters(xwidth=0.1, yheight=0.1, focus_aw=3.0, focus_ah=2.0, 
                      lambda0=wavelength, dlambda="0.1*wavelength")

In [ ]:
print(source)

In [ ]:
instrument.show_components()

There are a number of keyword arguments allowed when adding a component. These will mainly be discussed on the [component object page](component_object.ipynb), but a few are relevant for the instrument, because they handle in what order components are sequenced in the instrument. To illustrate this we add a slit and a guide to the instrument at reasonable positions. Notice these new components are added at the end of the instrument.

In [ ]:
slit = instrument.add_component("source_slit", "Slit", AT=2, RELATIVE=source)
slit.set_parameters(xwidth=0.015, yheight=0.015)

guide = instrument.add_component("guide", "Guide", AT=0.1, RELATIVE=slit)
guide.set_parameters(w1=0.03, h1=0.03, l=10.0)

In [ ]:
instrument.show_components()

The order of components is important in a McStas/McXtrace simulation as each will affect the ray state in the sequence shown with *print_components*. If one wants to add a component between the source and the slit, this can be done with the *before* or *after* keyword.

In [ ]:
monitor = instrument.add_component("PSD", "PSD_monitor", after="source")
monitor.set_AT(1.9, RELATIVE=source)
monitor.set_parameters(xwidth=0.1, yheight=0.1, filename='"PSD.dat"')

instrument.show_components()

The PSD monitor was inserted after the source, this could also be accomplished with the before keyword argument.
```
before="source_slit"
```
It is important to note that the McStas instrument file is read sequentially, so the position of the PSD monitor can not be relative to a later component, but must only refer to earlier components. At this point in development it is not possible to reorder components in the instrument object.

## Moving a component
It is also possible to move a component in the component sequence of the instrument. Lets add a *Lmonitor* at the end at move it to the PSD. The *move_component* command takes a name or component object to be moved, and then the same before and after keywords as *add_component*.

In [ ]:
Lmon = instrument.add_component("Lmon", "L_monitor", RELATIVE="guide")
instrument.show_components()

In [ ]:
instrument.move_component(Lmon, after=source)
instrument.show_components()

It is easy to introduce a mistake in an instrument by moving a component, as components uses each other as references for position and rotation. In the above example the *Lmon* was placed relative to the *guide* component, and thus McStasScript warns that an error was introduced when the component was moved, as *guide* has not been defined at the new position of *Lmon*. The method *check_for_errors* can be called for further information, here in a try block to catch the exception.

In [ ]:
try:
    instrument.check_for_errors()
except Exception as e:
    print(str(e))

## Removing a component
Components can be removed with the *remove_component* method. The input can be either a component name or a component object.

In [ ]:
instrument.remove_component(Lmon)
instrument.show_components()

## Making a component copy
It is possible to copy an existing component using the *copy_component* method. This can reduce both the amount of typing necessary, but also the risk of making a mistake. Here the guide is copied and placed a bit after the end of the first guide, with a small rotation.

In [ ]:
guide2 = instrument.copy_component("guide_2", "guide")
guide2.set_AT(guide.l + 0.01, RELATIVE=guide)
guide2.set_ROTATED([0, 0.5, 0], RELATIVE=guide)
print(guide2)

## Getting components
It is always possible to retrieve the component objects corresponding to components in the instrument with the *get_component* and *get_last_component* methods.

In [ ]:
my_source = instrument.get_component("source")
print(my_source)

In [ ]:
last_component = instrument.get_last_component()
print(last_component)

### Instrument diagram
McStasScript can generate a diagram of an instrument file to aid the user in understanding its content. Use the *show_diagram* method to display the figure. The legend shows how the different component categories are represented with different colors, and how AT and ROTATED is represented with arrows. The rest of the figure is the actual diagram of this instrument, showing the sequence of components.

If in a notebook and using the %matplotlib widget backend, hovering the mouse over the left side of the boxes show further information on the individual components.

The diagram will also show use of the keywords EXTEND, WHEN, JUMP and GROUP, as well as connections between Union components, though none of these are present in this diagram.

In [ ]:
instrument.show_diagram()

## Run the simulation
The simulation is executed with a call to the *backengine* method, which will return the generated data. If the simulation fails, the method returns None. McStasScript does check for some common mistakes before attempting to run the McStas simulation, if any problem is found a useful error message will be shown.

In [ ]:
data = instrument.backengine()

In [ ]:
print(data)

## Visualizing the instrument
It is possible to visualize the instrument using the visualization features in McStas / McXtrace. This is done using the *show_instrument* method that show the instrument with the currently set parameters. The method takes a format keyword, of which there are two allowed:

| Format | Description | |
| :-- | :-- | --- |
| webgl | 3D view in notebook or browser tab | (default) |
| window | 2D view in window | |


The default format webgl behaves differently whether in a notebook or from a script. In a notebook, the output will be shown directly in the cell as shown in the example below, but in a script it will open a new browser tab. If a new browser tab is desired even when running from a notebook, set the keyword argument *new_tab* to True.

When using the 3D view in webgl, use these controls to manipulate the view:

| Action | Effect on view |
| :-- | :-: |
| Hold left click and drag | Rotate |
| Hold right click and drag | Move |
| Hold mouse wheel and drag up/down | Zoom in/out |

In [ ]:
instrument.show_instrument()

The window format is a 2D view which is opened in a new window, and may not always work if one use McStasScript through the cloud or a docker container. It is better suited for getting measurements and ensuring the geometry is exactly as desired.

In [ ]:
instrument.show_instrument(format="window")

### Showing instrument file
McStasScript writes the instrument file for McStas in the process of running or visualizing the instrument. The file can be shown with the *show_instrument_file* method.

In [ ]:
instrument.show_instrument_file(line_numbers=True)

## Dump and load an instrument object
It is possible to save an instrument object to disk and load it later.

In [ ]:
instrument.dump("dump_file_name.dmp")

To load an instrument object from a file, use the *from_dump* method that takes the filename.

In [ ]:
loaded_instrument = ms.McStas_instr.from_dump("dump_file_name.dmp")

In [ ]:
loaded_instrument.show_components()
loaded_instrument.show_settings()